In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from scipy.ndimage import uniform_filter
from scipy.signal import windows
from scipy.fftpack import fft
from PIL import Image
from tqdm import tqdm

ROOT = Path.cwd().parent  # go to project root
DATA = ROOT / "data"
HEATMAP_TRAIN = DATA / "heatmaps" / "train"
HEATMAP_TEST  = DATA / "heatmaps" / "test"

for d in [HEATMAP_TRAIN, HEATMAP_TEST]:
    d.mkdir(parents=True, exist_ok=True)


In [2]:
c = 3e8                      # speed of light
fc = 60e9                    # 60 GHz radar
bw = 2e9                     # 2 GHz bandwidth (great for metal detection)
T_chirp = 1e-3               # 1 ms chirp duration
fs = 2.5e6                   # ADC sampling rate
N = int(T_chirp * fs)        # number of ADC samples per chirp
slope = bw / T_chirp         # chirp slope

def generate_chirp():
    t = np.arange(N) / fs
    chirp = np.exp(1j * 2 * np.pi * (fc*t + 0.5*slope*(t**2)))
    return t, chirp

t, base_chirp = generate_chirp()


In [4]:
def simulate_scene(targets, clutter_level=0.2, snr_db=20):
    rx = np.zeros(N, dtype=complex)
    for tg in targets:
        R = tg['range']
        v = tg['velocity']
        rcs = tg['rcs']

        # Delay
        tau = 2*R/c
        phase_delay = np.exp(-1j * 2 * np.pi * slope * tau * t)

        # Doppler shift
        doppler = np.exp(1j * 2 * np.pi * (2*v/c)*fc * t)

        rx += (rcs/(R+1e-6)) * (phase_delay * doppler)

    # Clutter (many tiny scatterers)
    for _ in range(int(100*clutter_level)):
        R = np.random.uniform(0.5, 8.0)
        v = np.random.uniform(-1.5, 1.5)
        rcs = np.random.uniform(0.01, 0.1)
        tau = 2*R/c
        rx += (rcs/(R+1e-6)) * np.exp(-1j * 2*np.pi * slope * tau * t)

    # Add noise
    signal_power = np.mean(np.abs(rx)**2)
    snr_linear = 10**(snr_db/10)
    noise_power = signal_power / snr_linear
    noise = np.sqrt(noise_power/2)*(np.random.randn(N)+1j*np.random.randn(N))
    rx += noise

    return rx


In [5]:
def generate_rd_map(targets, num_chirps=32, clutter=0.3, snr_db=20):
    frame = []
    for _ in range(num_chirps):
        rx = simulate_scene(targets, clutter_level=clutter, snr_db=snr_db)
        frame.append(rx)
    frame = np.array(frame)

    # Range FFT (fast time)
    win_r = windows.hann(N)
    range_fft = np.fft.fft(frame * win_r, n=4096, axis=1)
    range_fft = range_fft[:, :2048]

    # Doppler FFT (slow time)
    win_d = windows.hann(num_chirps)
    rd = np.fft.fftshift(np.fft.fft(range_fft * win_d[:, None], n=128, axis=0), axes=0)
    rd_power = np.abs(rd)**2
    rd_db = 10*np.log10(rd_power + 1e-12)

    return rd_db


In [6]:
def cfar(rd_map, guard=2, train=8, offset=5):
    cfar_map = np.zeros_like(rd_map, dtype=bool)
    rows, cols = rd_map.shape

    for r in range(train+guard, rows-train-guard):
        for c in range(train+guard, cols-train-guard):
            training_cells = rd_map[
                r-train-guard:r+train+guard,
                c-train-guard:c+train+guard
            ]
            guard_cells = rd_map[
                r-guard:r+guard,
                c-guard:c+guard
            ]

            noise = (np.sum(training_cells) - np.sum(guard_cells))
            noise /= (training_cells.size - guard_cells.size)

            threshold = noise + offset

            if rd_map[r, c] > threshold:
                cfar_map[r, c] = True

    return cfar_map


In [7]:
def save_heatmap(rd_map, out_path):
    img = np.max(rd_map, axis=0)
    img = (img - img.min()) / (img.max() - img.min() + 1e-12)
    img = (img * 255).astype(np.uint8)
    img = Image.fromarray(img).resize((224,224))
    img.save(out_path)


In [8]:
def generate_dataset(n=300, split="train"):
    out_dir = HEATMAP_TRAIN if split=="train" else HEATMAP_TEST

    labels = ["metal", "non_metal"]

    for i in tqdm(range(n)):
        label = labels[i % 2]

        if label == "metal":
            targets = [{
                "range": np.random.uniform(1,5),
                "velocity": np.random.uniform(-2,2),
                "rcs": np.random.uniform(1.0, 3.0)
            }]
            clutter = 0.3
        else:
            targets = [{
                "range": np.random.uniform(1,5),
                "velocity": np.random.uniform(-2,2),
                "rcs": np.random.uniform(0.05, 0.3)
            }]
            clutter = 0.5

        rd = generate_rd_map(targets, clutter=clutter)

        save_path = out_dir / label
        save_path.mkdir(exist_ok=True)

        save_heatmap(rd, save_path / f"{label}_{i}.png")

generate_dataset(400, "train")
generate_dataset(120, "test")


100%|██████████| 120/120 [00:08<00:00, 14.73it/s]
